In [2]:
# question

In [15]:
pd.set_option('display.max_columns', 200)

In [92]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
df = pd.read_csv('energydata_complete.csv')
df.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,T6,RH_6,T7,RH_7,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,17.166667,55.20,7.026667,84.256667,17.200000,41.626667,18.2,48.900000,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,17.166667,55.20,6.833333,84.063333,17.200000,41.560000,18.2,48.863333,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,17.166667,55.09,6.560000,83.156667,17.200000,41.433333,18.2,48.730000,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,17.166667,55.09,6.433333,83.423333,17.133333,41.290000,18.1,48.590000,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,17.200000,55.09,6.366667,84.893333,17.200000,41.230000,18.1,48.590000,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [93]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import make_pipeline
dff = df.drop(columns=['date','lights'])
scaler = MinMaxScaler()
normdf = pd.DataFrame(scaler.fit_transform(dff), columns=dff.columns)
x = normdf.drop(columns=['Appliances'])
y = normdf['Appliances']

In [94]:
x_train,x_test, y_train,y_test = train_test_split(x,y, test_size=0.30,random_state=42)
model = LinearRegression().fit(x_train,y_train)
pred = model.predict(x_test)
print('mae', mean_absolute_error(y_test,pred))

mae 0.0501337159005189


In [84]:
print('rmse', np.sqrt(mean_squared_error(y_test,pred)))

rmse 0.0875101567176056


In [86]:
print('r2score', r2_score(y_test,pred))

r2score 0.14898595722254737


In [87]:
def get_wieghts(model,feat, col_n):
    weights = pd.Series(model.coef_, feat.columns).sort_values()
    weights_df = pd.DataFrame(weights).reset_index()
    weights_df.columns = ['features', col_n]
    weights_df[col_n].round(3)
    return weights_df

linear_weight = get_wieghts(model, x_train, 'Multiple_linear_model_weight')

In [91]:
linear_weight.sort_values(by='Multiple_linear_model_weight',ascending= False)
# linear_weight

,features,Multiple_linear_model_weight
25,rv1,2.219382e+11
24,RH_1,5.536721e-01
23,T3,2.907042e-01
22,T6,2.364816e-01
21,Tdewpoint,1.178010e-01
20,T8,1.020164e-01
19,RH_3,9.595330e-02
18,RH_6,3.808434e-02
17,Windspeed,2.919433e-02
16,T4,2.896918e-02


#### answer 
RH_2, RH_1

In [98]:
model = Ridge(alpha=0.4).fit(x_train,y_train)
pred = model.predict(x_test)

In [99]:
print('rmse', np.sqrt(mean_squared_error(y_test,pred)))

rmse 0.08753385704628003


answer = NO

In [100]:
model = Lasso(alpha=0.001).fit(x_train,y_train)
pred = model.predict(x_test)

In [101]:
def get_wieghts(model,feat, col_n):
    weights = pd.Series(model.coef_, feat.columns).sort_values()
    weights_df = pd.DataFrame(weights).reset_index()
    weights_df.columns = ['features', col_n]
    weights_df[col_n].round(3)
    return weights_df

linear_weight = get_wieghts(model, x_train, 'Multiple_linear_model_weight')

In [102]:
linear_weight.sort_values(by='Multiple_linear_model_weight',ascending= False)

,features,Multiple_linear_model_weight
25,RH_1,0.017880
24,Windspeed,0.002912
14,RH_5,0.000000
23,rv2,-0.000000
22,RH_6,-0.000000
21,T2,0.000000
20,RH_2,-0.000000
19,T3,0.000000
18,RH_3,0.000000
17,T4,-0.000000


In [ ]:
# Anwser 26


In [103]:
rmse = np.sqrt(mean_squared_error(y_test,pred))
round(rmse, (3))

0.094